In [10]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.data.pocket_utils import get_atom_coordinates, find_pocket_atoms_RDKit
from src.data.utils import get_vdw_radius, mol2_to_rdkit_mol

In [11]:
def vdw_interactions(mol, atom_1, atom_2):
    """
    Determine if there is a van der Waals interaction between atom 1 and atom 2 based on whether the interatomic distance is less 
    than or equal to the sum of the van der Waals radii of the two atoms.
    
    Parameters:
    mol (Chem.Mol): The RDKit molecule.
    atom_1 (int): The index of the first atom.
    atom_2 (int): The index of the second atom.
    
    Returns:
    tuple: (0, 0, 0, 0, 0) if there is a van der Waals interaction, None otherwise 
    """

    # Get the coordinates of the atoms
    pos1 = np.array(mol.GetConformer().GetAtomPosition(atom_1))
    pos2 = np.array(mol.GetConformer().GetAtomPosition(atom_2))

    # Compute the distance between the two atoms
    distance = np.linalg.norm(pos1 - pos2)

    # Get the van der Waals radii of the two atoms
    atom_1_symbol = mol.GetAtomWithIdx(atom_1).GetSymbol()
    atom_2_symbol = mol.GetAtomWithIdx(atom_2).GetSymbol()
    vdw_radius_1 = get_vdw_radius(atom_1_symbol)
    vdw_radius_2 = get_vdw_radius(atom_2_symbol)

    print(f"Distance: {distance}")
    print(f"Sum of van der Waals radii: {vdw_radius_1 + vdw_radius_2}")
    

    # Check if the distance is less than or equal to the sum of the van der Waals radii
    if distance <= (vdw_radius_1 + vdw_radius_2):
        return (0, 0, 0, 0, 0)

    return None


In [12]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Define test cases
test_cases = [("CCCCCCCCCC=C", 0, 10), # expected output: None
              ("C=C", 0, 1) # expected output: (0, 0, 0, 0, 0)
              ]

# Initialize RDKit molecules and conformers
molecules = []
for smiles, a1, a2 in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)
    molecules.append((mol, a1, a2))

# Run test cases
for mol, atom_1, atom_2 in molecules:
    # Replace this with the actual function call
    result = vdw_interactions(mol, atom_1, atom_2)
    print(f"SMILES: {Chem.MolToSmiles(mol)}, Atoms: ({atom_1}, {atom_2}) -> Result: {result}")


Distance: 8.8289638298645
Sum of van der Waals radii: 3.4
SMILES: C=CCCCCCCCCC, Atoms: (0, 10) -> Result: None
Distance: 1.33466598359613
Sum of van der Waals radii: 3.4
SMILES: C=C, Atoms: (0, 1) -> Result: (0, 0, 0, 0, 0)


[17:59:13] Molecule does not have explicit Hs. Consider calling AddHs()
[17:59:13] Molecule does not have explicit Hs. Consider calling AddHs()


In [ ]:
test_mol = mol2_to_rdkit_mol("../../test_data/pdb/1a0q/1a0q_complex_charged.mol2")
